## Circuit cutting with automatic cut finding using the Circuit Knitting Toolbox

### Import relevant modules

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import EfficientSU2
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Options,
    Session,
    Sampler,
    RuntimeOptions,
)
from quantum_serverless import QuantumServerless

from circuit_knitting_toolbox.circuit_cutting import WireCutter

In [ ]:
serverless = QuantumServerless()

### Create a circuit to cut

In [ ]:
num_qubits = 8
circuit = EfficientSU2(
    num_qubits=num_qubits,
    reps=2,
    entanglement="linear",
    su2_gates=["ry"],
    insert_barriers=False,
)

shift = 0.25
params = (
    [np.pi / 2 - shift]
    + [shift]
    + [np.pi / 2 - shift]
    + [shift] * int(len(circuit.parameters) / 1 - 3)
)

circuit.assign_parameters(params, inplace=True)
circuit = circuit.decompose()

### Set up the Qiskit runtime service

In [ ]:
service_args = QiskitRuntimeService(
    channel="ibm_quantum",
    instance="YOUR_INSTANCE",
    token="YOUR_TOKEN",
).active_account()

### Find cuts that match our criteria

In [ ]:
# Set the Sampler and runtime options
# options = Options(resilience_level=1, optimization_level=3, execution={"shots": 8192})
# runtime_options = RuntimeOptions(backend="ibmq_qasm_simulator")

# Instantiate a WireCutter and decompose the circuit
# cutter = WireCutter(
#    qc, service_args=service_args, options=options, runtime_options=runtime_options
# )
backend_names = ["ibmq_kolkata", "ibm_auckland"]
cutter = WireCutter(circuit, service_args=service_args, backend_names=backend_names)  # Local Estimator

with serverless:
    cuts = cutter.decompose(
        method="automatic",
        max_subcircuit_width=6,
        max_cuts=2,
        num_subcircuits=[2],
    )

### Evaluate the subcircuits, then recompose the circuit and verify the error between the full and cut circuit distributions is within tolerance

In [ ]:
# Evaluate the subcircuits on backend
with serverless:
    subcircuit_instance_probabilities = cutter.evaluate(cuts)

# Recompose the circuit and generate the cut circuit's probability distribution
with serverless:
    reconstructed_probabilities = cutter.recompose(
        subcircuit_instance_probabilities, cuts, num_threads=4
    )

# Use a statevector simulator to calculate the error between the inferred and actual distributions
metrics = cutter.verify(reconstructed_probabilities)
print(metrics)